<a href="https://colab.research.google.com/github/Elwing-Chou/tibaml1027/blob/main/imdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

dataset = tf.keras.utils.get_file(
    fname="aclImdb.tar.gz", 
    origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
    extract=True,
)

84140032/84125825 [==============================] - 2s 0us/step


In [4]:
dataset

'/root/.keras/datasets/aclImdb.tar.gz'

In [2]:
import os
import glob
import pandas as pd
def getdata(mid):
    dn = os.path.dirname(dataset)
    posfn = glob.glob(os.path.join(dn, "aclImdb", mid, "pos", "*"))
    negfn = glob.glob(os.path.join(dn, "aclImdb", mid, "neg", "*"))
    contents = []
    for fn in posfn + negfn:
        with open(fn, encoding="utf-8") as f:
            contents.append(f.read())
    df = pd.DataFrame({
        "content":contents,
        "sentiment":[1] * len(posfn) + [0] * len(negfn)
    })
    return df

In [5]:
train_df = getdata("train")
test_df = getdata("test")

In [7]:
test_df

,content,sentiment
0,"Valentine ""Dogkiller"" Dussaut and Joe ""The Jud...",1
1,Ruthless mercenary Bruno Rivera (Paul Naschy i...,1
2,Should we take the opening shot as a strange f...,1
3,"Being a retired medical/health field ""toiler i...",1
4,I wasn't expecting a great deal from this film...,1
...,...,...
24995,"I give it a 2, because of the beautiful Medite...",0
24996,How bad can you make a film. A good question w...,0
24997,"Oh, my. Oh, this is a *really* bad movie. The ...",0
24998,Apparently re-cut episodes from the Gangbuster...,0


In [13]:
# Tokenize: 把你的詞變成數字
from tensorflow.keras.preprocessing.text import Tokenizer
tok = Tokenizer(num_words=3000)
tok.fit_on_texts(train_df["content"])

In [12]:
# tok.word_index
# tok.index_word

In [14]:
# Sequence: 化成數字的序列
x_train_seq = tok.texts_to_sequences(train_df["content"])
x_test_seq = tok.texts_to_sequences(test_df["content"])

In [18]:
# pd.DataFrame(x_train_seq)
# Padding: 截長補短變成一樣長
from tensorflow.keras.preprocessing.sequence import pad_sequences
x_train_pad = pad_sequences(x_train_seq, maxlen=512)
x_test_pad = pad_sequences(x_test_seq, maxlen=512)
pd.DataFrame(x_train_pad)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,301,117,1,15,1,751,26,1983,46,581,51,26,119,46,154,20,53,8,160,18,21,14,49,14,1,83,19,18,2,448,77,354,9,1474,31,324,1301,7,7,1174
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2425,34,53,147,2,171,466,1,62,8,3,2588,4,5,709,2006,53,1883,111,2,57,1492,3,618,16,1016,574,2095,7,7,924,8,257,3,1036,898,5,69,130,2353
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,104,1906,15,1848,3,752,12,1025,5,987,5,196,91,872,2,1,2064,9,213,731,55,2,141,27,821,9,77,590,148,47,431,23,54,286,369,50,4,95,786,155
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,881,2888,1,129,122,2,414,80,3,191,4,1,127,7,7,42,35,12,9,817,627,53,109,3,173,4,250,463,3,1015,274,1500,3,285,763,6,3,52,315,17
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,88,4,9,49,1153,718,92,45,54,14,2473,1038,41,157,19,42,1,240,4,17,22,67,866,142,2,354,14,193,14,22,89,94,1,1318,4,533,96,73,7,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,31,1338,89,614,866,70,372,5,1,687,7,7,1209,36,1,336,860,1,113,215,1,115,342,405,3,1498,236,14,124,1,4,1,174,441,3,1386,7,7,339,155
24996,9,66,282,36,1,1,776,4,1,1372,1,585,4,32,2,12,474,133,12,163,22,920,12,131,1907,380,967,472,9,13,29,15,91,55,36,130,5,172,1006,1722,...,283,15,256,2,1,474,1171,2081,12,1,4,9,66,122,231,100,1,19,645,9,13,18,21,1,84,1372,44,413,91,831,8,1,55,4,434,7,7,1239,43,4
24997,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,234,26,44,74,8,46,181,539,105,2,26,13,3,181,1254,281,352,54,28,141,435,65,55,20,11,19,10,700,140,4,9,2,10,89,232,12,10,1046,3,152
24998,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1386,8,257,706,385,29,11,44,6,798,136,16,1,16,908,321,136,2,798,50,538,2,564,533,12,163,9,52,1330,17,2,10,2304,383,29,1,99,4,11,429
